In [1]:
import pandas as pd
import numpy as np
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import SVDpp
from surprise import Dataset
from surprise.model_selection import GridSearchCV
from sklearn.preprocessing import normalize
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from collections import defaultdict
import torch
from torch import nn
from scipy.sparse import rand as sprand
from sklearn.preprocessing import LabelEncoder

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 232 from PyObject
<frozen importli

In [3]:
def read(args):
    if args.name == 'Frappe':
        df = pd.read_csv('frappe.csv', sep="\t")
        meta_app = pd.read_csv('meta.csv', sep="\t")
        df_context = df.copy()
        del df_context['cost']
        df_context['cnt'] = df['cnt'].astype('float')
        df_context['cnt'] = df_context['cnt'].apply(np.log10) ## transformation of the number of interactions by log
        a = df_context['user'].value_counts()[df_context['user'].value_counts() >= 10] ## Here we apply the 10-core setting to ensure the quality of the dataset
        df_context = df_context[df_context['user'].isin(list(a.index))]
        
        
        meta_app = meta_app[meta_app['item'].isin(list(set(df_context['item'].values)))] ## Here we filter out the items that are excluded by the 10-core setting
        
        
        
        ### From line 18 to line 24, we give new index to each user and item after the 10-core setting
        le = LabelEncoder()
        b = le.fit_transform(meta_app['item'])
        i = meta_app['item'].valuesdict_co = dict(zip(i, b))
        df_context['item'] = df_context['item'].map(dict_co)
        meta_app['item'] = b 
        b = le.fit_transform(df_context['user'])
        df_context['user'] = b
        
        
        df_context = trans_frame(df_context,3) #we give index to all the contextual conditions (starting from 0)
        
        
        
        ### From line 32 to line 39 we process the attribute of apps namely category, number of downloads, price and ratings given by other users
        meta_app_knowledge = meta_app[['item','category','downloads','language','price','rating']]
        meta_app_knowledge['price'] = np.where(meta_app_knowledge['price'] == 'Free', 'Free',
                                       np.where(meta_app_knowledge['price'] == 'unknown','Unknown','Paid'))
        meta_app_knowledge['rating'][meta_app_knowledge['rating'] == 'unknown'] = 0
        meta_app_knowledge['rating'] = meta_app_knowledge['rating'].astype('float')
        meta_app_knowledge['rating'] = np.where(meta_app_knowledge['rating'] == 0, 'Unknown',
                                       np.where(meta_app_knowledge['rating'] >=4.3, 'High_price',
                                               np.where(meta_app_knowledge['rating'] >= 3.8, 'Mid_price','Low_price')))
        
        
        meta_app_knowledge = trans_frame(meta_app_knowledge,1)#we give index to all attributes of items (starting from 0)
        meta_app_knowledge.to_csv('meta_app.csv',index = False)
        df_context.to_csv('final_app.csv', index = False)

In [4]:
def trans_frame(df,k):
    x = 0
    for col in df.columns[k:]:
        num = len(df[col].value_counts())
        le = LabelEncoder()
        y = le.fit_transform(df[col])
        df[col] = y
        df[col] = df[col] + x
        x += num
    return df